In [1]:
# 리뷰 리스트 불러오기
import json
with open('./res/reviews.json','r', encoding='UTF8') as f:
    review_list = json.load(f) 

review_list[:3]

[{'review': '공덕역 1번 출구에서 도보 1~2분 정도 거리에 위치해서 접근성이 좋아요. 체크인 시 응대해 준 직원분은 다소 사무적이고 대단히 친절하지는 않았지만 신속한 처리 덕분에 빠르게 객실로 올라올 수 있었어요. 그날그날 객실 상황에 따라 샤워부스/욕조형 객실을 선택할 수 있는것 같고, 고층을 배정받아 멀리 한강과 63스퀘어 빌딩도 볼 수 있었네요. 객실엔 딱 필요한 것만 잘 갖추어져 있어서 비즈니스 호텔로서 기능은 충분하고, 1층에 세븐일레븐 편의점이 있어서 이것저것 사기 편했습니다. 재방문 의사 있어요~',
  'stars': 5,
  'date': '2023.09.24'},
 {'review': '피트니스센터가 아주 잘되어있어서 좋았습니다\r\n숙소도 트윈밖에 남아있지 않았는데 더블을 원하니깐 최대한 맞춰주려고 노력하주셔서 감사했습니다!',
  'stars': 5,
  'date': '2023.01.26'},
 {'review': '설 연휴 호캉스차 숙소를 알아보다가 비교 후 예약한 곳인데 후기가 너무 극과 극 천차만별이라 첨엔 반신반의 확신이 없었어요,, 근데 머물러보니 호텔 경험 많지 않은 저에게는 청소가 엉망이다, 친절하지 않다 등의 별점 낮은 후기와 같은 일은 없었습니다!\n당일 랜덤 숙소권인데 체크인 시 요청사항 여쭤보시고는 2개 맞춰주셨어요 ㅎㅎ (요청사항: 고층, 넓은 원배드, 욕조 등) \n룸 컨디션 좋았고 패브리즈와 마스터 키 요청 모두 친절히 추가로 응해주셨습니다\n서울 한복판에 가성비 호텔로 좋습니다\n퇴실 전 아침에 준비할 때 찍은 사진 함께 공유해요 ~~',
  'stars': 5,
  'date': '2023.01.26'}]

In [2]:
#리뷰 오전과 그외 구별해서 리스트에 저장
good_cnt, bad_cnt=0,0

for review in review_list: 
    if review['stars'] == 5: 
        good_cnt += 1
    else:
        bad_cnt += 1 

good_cnt, bad_cnt

(172, 48)

In [3]:
#리뷰 5점과 그외를 구별해서 각각의 리스트에 넣기 근데 리뷰 시작점과 끝점을 구별 할 수 있게 [REVIEW_START] [REVIEW_END]를 넣음
reviews_good, reviews_bad = [], []  

for review in review_list:
    if review['stars'] == 5:
        reviews_good.append('[REVIEW_START]'+ review['review' ] +'[REVIEW_END]')
    else: 
        reviews_bad.append('[REVIEW_START]' + review['review'] + '[REVIEW_END]')

reviews_bad[:3]

['[REVIEW_START]주차불편 도보이용 외에는 다 좋았어요[REVIEW_END]',
 '[REVIEW_START]좋았어요 무냐히 사용합니다[REVIEW_END]',
 '[REVIEW_START]1. 신라스테이는 여름에 가지 않는걸 추천드립니다. 중앙제어 냉방 시스템이기 때문에, 묵었던 날이 폭염이었음에도 불구하고 프론트에 연락해도 아무 것도 해줄 수 없다는 말만 들었습니다. 요즘 시대에 아직도 중앙제어 온도 기능이 있다는게 놀랍습니다. 그래도 친절한 직원분들 덕분에 선풍기 하나는 받아서 잠은 잤습니다. 이건 서비스가 아니라 구조적인 문제라고 생각되네요.\n\n2. 핸드폰이 침대 옆에 떨어져 줍다가 다리가 많은 돈벌레를 보고 놀랐습니다. 저는 그나마 벌레를 잘 잡는 편이라 그 자리에서 잡고 다음 날 클레임도 안하고 대수롭지 않게 넘어갔지만 벌레를 혐오하거나 무서워하는 분들이 봤어도 이렇게 조용히 넘어갔을까 하는 생각이 듭니다.\n\n그동안 신라스테이 광화문, 서대문, 구로, 삼성, 마포 등 서울 내 지점들을 이용한 횟수만 30회 가까이 되는데 앞으로는 조금 더 투자해서 다른 곳에 묵을 예정입니다. 제가 이 정도 비용을 지불하고, 벌레 안 나오는 시원한 곳에서 편하게 묵으려고 한게 과한 욕심이었을까요...?[REVIEW_END]']

In [4]:
#특정 시간동안에 (여기는 6개월) 남긴 리뷰만을 리스트에 넣어놓게 해준 것 
import datetime
from dateutil import parser

def preprocess_reviews(path='./res/reviews.json'):
    with open(path, 'r',encoding='UTF-8') as f:
        review_list = json.load(f)
    
    reviews_good, reviews_bad = [], []

    current_date = datetime.datetime.now()
    date_boundary = current_date - datetime.timedelta(days=6*30)

    for review in review_list:
        review_date_str = review['date']
        try:
            review_date = parser.parse(review_date_str) 
        except(ValueError, TypeError):
            review_date = current_date 

        if review_date < date_boundary:
            continue
        
        if review['stars'] == 5:
            reviews_good.append('[REVIEW_START]' + review['review'] + '[REVIEW_END]')
        else:
            reviews_bad.append('[REVIEW_START]' + review['review'] + '[REVIEW_END]')
        
    review_good_text = '\n'.join(reviews_good)
    review_bad_text = '\n'.join(reviews_bad) 
    
    return review_good_text, review_bad_text

good, bad = preprocess_reviews()
good[:100]

'[REVIEW_START]너무 깨끗하고 좋아요[REVIEW_END]\n[REVIEW_START]너무 깨끗하고 좋아요[REVIEW_END]\n[REVIEW_START]너무 깨끗하고 좋아'

In [5]:
# 두 ai보조자들을 만들어낸다. 서로를 비교하여, 사용자 리뷰에 대한 요약의 품질을 평가합니다. 평가 기준은 유용성, 정확성, 깊이 등을 포함하며, AI 모델은 이러한 기준에 따라 최종 결정을 내립니다. 
def pairwise_eval(reviews, answer_a, answer_b):
    eval_prompt = f"""[System]
Please act as an impartial judge and evaluate the quality of the Korean summaries provided by two
AI assistants to the set of user reviews on accommodations displayed below. You should choose the assistant that
follows the user’s instructions and answers the user’s question better. Your evaluation
should consider factors such as the helpfulness, relevance, accuracy, depth, creativity,
and level of detail of their responses. Begin your evaluation by comparing the two
responses and provide a short explanation. Avoid any position biases and ensure that the
order in which the responses were presented does not influence your decision. Do not allow
the length of the responses to influence your evaluation. Do not favor certain names of
the assistants. Be as objective as possible. After providing your explanation, output your
final verdict by strictly following this format: "[[A]]" if assistant A is better, "[[B]]"
if assistant B is better, and "[[C]]" for a tie.
[User Reviews]
{reviews}
[The Start of Assistant A’s Answer]
{answer_a}
[The End of Assistant A’s Answer]
[The Start of Assistant B’s Answer]
{answer_b}
[The End of Assistant B’s Answer]"""
    
    completion = client.chat.completions.create(
        model='gpt-4o-2024-05-13',
        messages=[{'role': 'user', 'content': eval_prompt}],
        temperature=0.0
    )

    return completion

In [6]:
PROMPT_BASELINE = f"""아래 숙소에 대해서 5문장 내로 요약해줘:""" 

In [7]:

from openai import OpenAI
import os 
from dotenv import load_dotenv

load_dotenv()

True

In [8]:
reviews, _ = preprocess_reviews(path="./res/reviews.json") #6개월 이내 좋은 리뷰 불러오기
client = OpenAI(api_key = os.getenv('OPENAI_API_KEY')) # openai 불러오기

def summarize(reviews,prompt,temperature=0.0,model='gpt-3.5-turbo-0125'):
    prompt = prompt + '\n\n' + reviews #AI 모델에게 작업을 요청하는 텍스트 내용

    #prompt를 모델에 전달하고 요약을 요청
    completion = client.chat.completions.create(
        model=model,
        messages=[{'role': 'user', 'content': prompt}],
        temperature=temperature
    )

    return completion 

print(summarize(reviews, PROMPT_BASELINE).choices[0].message.content)

깨끗하고 친절한 직원들이 있는 공덕역 근처의 숙소로, 편안한 휴식을 취할 수 있는 곳이며, 시설과 청결도 모두 만족스러운 곳입니다.


In [9]:
eval_count = 10

summaries_baseline = [summarize(reviews, PROMPT_BASELINE,temperature=1.0).choices[0].message.content for _ in range(eval_count)]
summaries_baseline

['깨끗하고 친절한 직원들이 인상적인 숙소로, 교통도 편리하며 휴식하기 좋았고, 시설은 만족스러웠으며 청결도도 좋았다.',
 '깨끗하고 친절한 직원들이 좋은 숙소, 공덕역 근처에 위치하여 교통이 편리하며, 피트니스 시설 조금 불만족스러운 부분이 있지만 전반적으로 만족스러운 곳이었습니다.',
 '깨끗한 숙소, 친절한 직원, 편리한 위치로 만족하는 고객이 많은 숙소입니다. 다만 객실 청결이 부족한 점과 일부 시설의 노후화가 조금 아쉬운 부분이 있습니다.',
 '깨끗하고 친절한 서비스로 만족스러운 숙박을 경험할 수 있는 신라스테이 숙소입니다. 위치가 편리하고 침구가 편안하며 조식도 맛있어서 다시 방문하고 싶은 곳입니다. 피트니스 시설은 조금 협소하지만 전반적으로 만족스러운 편입니다. 더블베드 이불에 털이 있는 등 몇가지 아쉬운 부분이 있지만, 숙박 경험은 좋았습니다.',
 '깨끗하고 친절한 직원분들이 좋아하여 항상 잘 이용한다. 위치도 편리하고 시설도 만족스럽다.',
 '깨끗하고 친절한 직원이 있는 여의도 출장 숙소이며, 시설은 만족스럽지만 방음에 조금 민감해하는 이용자도 있음을 언급하고 있습니다.',
 '깨끗하고 친절, 가격대비 만족, 교통 편리, 호캉스 추천, 친절한 직원들.',
 '위치가 편리하고 깨끗하며 직원들이 친절한 신라스테이 숙소에 대한 리뷰들입니다.',
 '깨끗하고 친절한 직원들이 있는 여의도 출장 숙소로 만족도가 높은 편이며, 동급 숙소에서도 무난한 편이라고 한다.',
 '신라스테이는 깨끗하고 만족스럽게 묵을 수 있는 숙소로, 공덕역 근처에 위치하고 있습니다. 직원들의 친절한 응대와 편안한 휴식을 취할 수 있는 환경이 자랑이며, 조식도 맛있게 즐길 수 있습니다. 다만 몇몇 시설이 노후화되어 업그레이드가 필요한 부분도 있습니다.']

In [10]:
summary_real_20241003 = '교통이 편리한 위치에 있으며, 청결한 상태를 유지하고 있어 많은 후기가 긍정적입니다. 객실은 넷플릭스 시청을 위한 기기 대여가 필요하지만 전반적으로 깔끔하고 편안한 분위기를 제공합니다. 직원들의 친절한 응대가 인상적이며, 조식과 디너 서비스 역시 우수하다는 평이 있습니다. 다만, 방음과 시설 노후화 관련 아쉬운 점이 언급되기도 했습니다. 종합적으로 접근성과 서비스가 우수한 조건을 갖춘 숙소입니다.'

In [17]:

from tqdm import tqdm

def pairwise_eval_batch(reviews, answers_a, answers_b):
    a_cnt,b_cnt,draw_cnt=0,0,0 
    for i in tqdm(range(len(answers_a))):
        #reviews와 두 개의 요약 결과 answers_a[i], answers_b[i]를 비교 평가
        # 모델 전체 응답이 저장됨
        '''예시:
        completion = {
    "choices": [
        {
            "message": {
                "role": "assistant",
                "content": """
                Based on the provided user reviews, Assistant A’s answer provides better overall accuracy and relevance.
                [[A]]
                """
            }
        }
    ]
}
        '''
        completion = pairwise_eval(reviews, answers_a[i], answers_b[i]) 

        #텍스트 내용 저장
        '''예시:
        verdict_text = """Based on the provided user reviews, Assistant A’s answer provides better overall accuracy and relevance.
        [[A]]
        
        """
        '''
        verdict_text = completion.choices[0].message.content 

        if '[[A]]' in verdict_text:
            a_cnt += 1
        elif '[[B]]' in verdict_text:
            b_cnt += 1
        elif '[[C]]' in verdict_text:
            draw_cnt += 1
        else: 
            print("Evaluation Error")

    return a_cnt, b_cnt, draw_cnt

wins, loses, ties = pairwise_eval_batch(reviews, summaries_baseline, [summary_real_20241003]*eval_count) 
print(f"Wins: {wins}, Loses: {loses}, Ties: {ties}")

100%|██████████| 10/10 [00:44<00:00,  4.48s/it]

Wins: 2, Loses: 8, Ties: 0


In [12]:
prompt = f"""당신은 요약 전문가입니다. 사용자 숙소 리뷰들이 주어졌을 때 요약하는 것이 당신의 목표입니다.

요약 결과는 다음 조건들을 충족해야 합니다:
1. 모든 문장은 항상 존댓말로 끝나야 합니다.
2. 숙소에 대해 소개하는 톤앤매너로 작성해주세요.
  2-1. 좋은 예시
    a) 전반적으로 좋은 숙소였고 방음도 괜찮았다는 평입니다.
    b) 재방문 예정이라는 평들이 존재합니다.
  2-2. 나쁜 예시
    a) 좋은 숙소였고 방음도 괜찮았습니다.
    b) 재방문 예정입니다.
3. 요약 결과는 최소 2문장, 최대 5문장 사이로 작성해주세요.
    
아래 숙소 리뷰들에 대해 요약해주세요:"""

eval_count = 10
summaries = [summarize(reviews, prompt, temperature=1.0).choices[0].message.content for _ in range(eval_count)]
wins, losses, ties = pairwise_eval_batch(reviews, summaries, [summary_real_20241003 for _ in range(len(summaries))])
print(f'Wins: {wins}, Losses: {losses}, Ties: {ties}')

100%|██████████| 10/10 [00:31<00:00,  3.15s/it]

Wins: 1, Losses: 9, Ties: 0


In [18]:

def preprocess_reviews(path='./res/reviews.json'):
    #리뷰 리스트 불러오기
    with open(path, 'r',encoding='UTF-8') as f:
        review_list = json.load(f)

    #5점 리뷰와 그외 리뷰 리스트
    reviews_good, reviews_bad = [], []

    #리뷰 시간 설정(6개월 이내)
    current_date = datetime.datetime.now()
    date_boundary = current_date - datetime.timedelta(days=6*30)

    #6개월 내인 리뷰와 리뷰 내용이 30자 이상인 리뷰를 찾고 별점에 따라 리스트 분리
    for review in review_list:
        review_date_str = review['date']
        try:
            review_date = parser.parse(review_date_str) 
        except(ValueError, TypeError):
            review_date = current_date 

        if review_date < date_boundary:
            continue
        
        if len(review['review']) < 30:
            continue

        if review['stars'] == 5:
            reviews_good.append('[REVIEW_START]' + review['review'] + '[REVIEW_END]')
        else:
            reviews_bad.append('[REVIEW_START]' + review['review'] + '[REVIEW_END]')
        
    review_good_text = '\n'.join(reviews_good)
    review_bad_text = '\n'.join(reviews_bad) 
    
    return review_good_text, review_bad_text
#좋은 리뷰 데이터를 처리하여 reviews 변수에 저장
reviews, _ = preprocess_reviews(path="./res/reviews.json")

In [14]:
eval_count = 10 #요약 수행 갯수 설정

#summarize 함수를 통해 요약 결과를 생산. choices[0].message.content를 통해 추출
summaries = [summarize(reviews, prompt, temperature=1.0).choices[0].message.content for _ in range(eval_count)]

#실제요약과 비교하여 평가 
wins, losses, ties = pairwise_eval_batch(reviews, summaries, [summary_real_20241003 for _ in range(len(summaries))])
print(f'Wins: {wins}, Losses: {losses}, Ties: {ties}')

100%|██████████| 10/10 [00:36<00:00,  3.63s/it]

Wins: 6, Losses: 4, Ties: 0


In [19]:
reviews_1shot, _ = preprocess_reviews(path="./res/shillastay_Seodaemun.json") 
summary_1shot = summarize(reviews_1shot, prompt, temperature=1.0, model = 'gpt-4o-2024-05-13').choices[0].message.content

print(summary_1shot)

신라스테이 서대문은 서대문역 바로 앞이라는 뛰어난 위치와 친절한 직원들로 많은 호응을 받고 있는 숙소입니다. 객실은 깔끔하고 아늑하며, 방음과 청결에 대한 다양한 의견이 있지만 전반적으로 만족하시는 편입니다. 조식 메뉴는 다양하고 맛있다는 평이 많습니다. 주차는 인근 주차장을 이용할 수 있으며, 교통 접근성이 매우 좋아 타지역에서 오시는 분들에게 추천드릴 만합니다. 재방문 의사가 있다는 긍정적인 후기들도 다수 있습니다.


In [20]:
reviews_1shot, _ = preprocess_reviews(path="./res/shillastay_Seodaemun.json") 
summary_1shot = summarize(reviews_1shot, prompt, temperature=1.0, model = 'gpt-4o-2024-05-13').choices[0].message.content
prompt_1shot = f"""당신은 요약 전문가입니다. 사용자 숙소 리뷰들이 주어졌을 때 요약하는 것이 당신의 목표입니다.

요약 결과는 다음 조건들을 충족해야 합니다:
1. 모든 문장은 항상 존댓말로 끝나야 합니다.
2. 숙소에 대해 소개하는 톤앤매너로 작성해주세요.
  2-1. 좋은 예시
    a) 전반적으로 좋은 숙소였고 방음도 괜찮았다는 평입니다.
    b) 재방문 예정이라는 평들이 존재합니다.
  2-2. 나쁜 예시
    a) 좋은 숙소였고 방음도 괜찮았습니다.
    b) 재방문 예정입니다.
3. 요약 결과는 최소 2문장, 최대 5문장 사이로 작성해주세요.

다음은 리뷰들과 요약 예시입니다.
예시 리뷰들:
{reviews_1shot}
예시 요약 결과:
{summary_1shot}
    
아래 숙소 리뷰들에 대해 요약해주세요:"""

summaries = [summarize(reviews, prompt, temperature=1.0, model='gpt-3.5-turbo-0125').choices[0].message.content for _ in range(eval_count)]
wins, losses, ties = pairwise_eval_batch(reviews, summaries, [summary_real_20241003 for _ in range(len(summaries))])
print(f'Wins: {wins}, Losses: {losses}, Ties: {ties}')
print(summaries)

100%|██████████| 10/10 [00:36<00:00,  3.64s/it]

Wins: 8, Losses: 2, Ties: 0
['전반적으로 좋은 평가들을 받은 숙소입니다. 방음이 조금 아쉽다는 평이 있지만, 입지 조건은 좋았고 편안한 숙박이었다고 합니다. 침구는 따뜻하고 편안하며, 조식도 맛있었다고 합니다. 응대해주신 직원 분들이 친절하셨고, 깨끗한 숙소라는 점이 좋았습니다. 다만, 시설이 노후화되어 업그레이드가 필요하다는 의견도 있습니다.요약 결과: 전반적으로 좋은 평가를 받은 숙소이며, 방음이 조금 아쉬웠으나 입지 조건이 좋았고 편안한 숙박 경험을 제공하였습니다. 침구는 편안하고 조식도 맛있었으며, 응대해주신 직원분들이 친절하셨고 깨끗한 숙소였습니다.러나, 시설이 노후화되어 업그레이드가 필요하다는 의견이 있습니다.', '전체적으로 좋은 평가를 받았습니다. 방음은 아쉬웠지만 입지조건이 좋았다는 평이 있었습니다. 침구는 따뜻하고 편안하며 조식도 맛있었고, 직원 분들의 응대가 친절했습니다. 다만 몇몇 시설이 노후화되어 업그레이드가 필요한 점이 언급되었습니다. 전반적으로 청결하고 편안한 숙박을 즐길 수 있었습니다.', '전반적으로 좋은 숙소였고, 방음이 살짝 아쉬운 점이 있습니다. 위치도 좋고, 편하게 쉬다갑니다. 침구는 따뜻하고 편안하며, 조식도 맛있어 요긴하게 이용했습니다. 직원분들의 친절한 응대도 기억에 남을 만큼 좋았습니다. 숙소는 깔끔하고, 다시 방문할 의향이 있습니다.', '복도 방음이 조금 아쉽지만, 입지 조건은 좋고, 잘 있다 갑니다.\n침구는 편안하고 따뜻하며, 일과 휴식을 동시에 할 때 이용하기 좋습니다. 다만, 액자가 삐뚤어져 있거나 욕실 내 바닥 미끄럼 주의 팻말이 틀어져 있는 등 관리가 덜 된 점이 아쉽습니다.\n역 근처에 위치하고 교통이 편리하며, 응대해주신 직원들이 매우 친절하셔서 즐거운 여행이었고, 숙소는 깔끔했습니다.\n숙소는 깨끗하고 만족스럽지만 시설이 조금 노후화되어 업그레이드가 필요합니다.\n전반적으로 숙소는 깔끔하고 인프라가 좋으며 직원분들도 친절하셨습니다. 늦은 시간 예약에도 친절히 잘 해주시고, 